# Do UFO Shapes vary regionally? A Data Science Exploration

In [ ]:
import pandas as pd
import numpy as np
from kgen2.BERTcruncher.context_vecs import *

PATH = DPATH+'ufo/'
df = pd.read_csv(PATH + 'sightings.csv')

print(PATH, len(df))

level=0
W_ = ['ship', 'ufo', 'craft']

#(1) Set up .csv file for data repo
tweet_data = pd.DataFrame(columns=['state', 'shape', 'w', 'vec'])
tweet_data.to_csv(PATH+'ufo-vecs.csv', index=False, encoding='utf-8')

#(2) Generate embeddings with appropriate metadata
for name,r,TEXT in df[['state', 'shape', 'text']].values:
    vecs = []

    for w_ in W_:
        if w_ in TEXT.lower():
            try:
                vecs = nC(str(w_),str(TEXT),layer_no=level)
                update = [[name, r, w_, str(vec.view(-1).tolist())] for vec in vecs]
                update = pd.DataFrame(np.array(update), columns=list(tweet_data))
                update.to_csv(PATH +'ufo-vecs.csv', index=False, encoding='utf-8', header=False, mode='a')
            except ValueError:
                0
            except IndexError:
                0

In [ ]:
import pyjags
import pandas as pd
import numpy as np

DATAPATH = '/home/zaq/d/ufo/'
df = pd.read_csv(DATAPATH+'shape_counts.csv')
df['shape'], df['state'] = df['shape'].fillna('unknown'), df['state'].fillna('UNK')

A = df['abduction'].values

cdic = {shape:i for i, shape in enumerate(df['shape'].unique())}
cat = np.array([cdic[shape] for shape in df['shape'].values])
dicc = {i:shape for shape, i in cdic.items()}

sdic = {state:i for i, state in enumerate(df['state'].unique())}
sta = np.array([sdic[state] for state in df['state'].values])
dics = {i:state for state, i in sdic.items()}

inputs = {'A': A,
          'C': cat, 'Cm': cat.max(),
          'S': sta, 'Sm': sta.max(),
          'IDX': len(df)
          }

m = """
model {
    for (c in 1:Cm){
        CRAFT[c] ~ dunif(0,100)
    }

    for (s in 1:Sm){
        STATE[s] ~ dunif(1,10)
    }

    for (i in 1:IDX){
        A ~ dpois( (CRAFT[C[i]] * STATE[S[i]]) / N )
    }

    N ~ dunif(1,IDX*2)

}
"""

Nsamples = 1000
chains = 3
burn_in = 1000

varnames = [
    'CRAFT',
    'STATE',
    'N'
]

model = pyjags.Model(m, data=inputs, chains=chains)

model.sample(burn_in, vars=[])

samples = model.sample(Nsamples, vars=varnames)
for i in varnames:
    x = samples[i].reshape(*samples[i].shape[:-2], samples[i].shape[-2]*samples[i].shape[-1])
    np.save(DATAPATH+i+'.npy', x, allow_pickle=False)

    print(i, x.shape)